In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
# import miniball
import numpy as np

import sys
sys.path.append('../')
import dyrect as dy
from dyrect import draw_complex, unit_circle_sample, EpsilonNet, WitnessComplex, PatchedWitnessComplex

In [ ]:
np.random.seed(2)

## generate a point cloud of a shape '8'
# crc1 = unit_circle_sample(4000, 0.75) + [1.1,0]
# crc2 = unit_circle_sample(4000, 0.75) - [1.1,0]
# points = np.append(crc1, crc2, axis=0)
# eps=.25

## generate a square point cloud 
points = np.random.random((10000,2))
eps=0.1

EN = EpsilonNet(eps, 0)
EN.fit(points)
lms = EN.landmarks
# points = np.random.random((10000,2))

print("EN done")

# plt.figure()
# plt.scatter(points[:,0], points[:,1])
# plt.show()

In [ ]:
# fig = plt.figure(figsize=(15,10))
# fig = plt.figure()
fig = plt.figure(figsize=(16, 6))
rows = 1
cols = 2

ax = plt.subplot(rows, cols, 1)
plt.scatter(points[:,0], points[:,1], s=0.2)
plt.scatter(lms[:,0], lms[:,1], s=5.2)

for lm in lms:
    crc = plt.Circle(lm, eps, color='r', alpha=0.05)
    ax.add_patch(crc)


ax = plt.subplot(rows, cols, 2)
anc = WitnessComplex(lms, points, 2)
draw_complex(anc, fig=fig, ax=ax, vlabels=True)
non_witnesses = np.array([points[i, :] for i in anc._barren_witnesses[2]])
# print(anc.not_witnessed[2])
ax.scatter(non_witnesses[:, 0], non_witnesses[:, 1], color='k', s = 0.5)
plt.show()

In [ ]:
### Draw (dim)th order areas of influence for landmarks with indices in the set verts
# verts = [4, 30, 33, 44, 57]
verts = [22, 23, 34, 52, 60, 62, 67]
# verts = [3, 8, 17, 36, 37, 40, 47]
# verts = [5, 25, 51, 54]

dim = 2

from scipy.spatial.distance import cdist
from itertools import combinations

edge_wareas = dict()
edge_witnesses = dict()
triangle_wareas = dict()
triangle_witnesses = dict()


vcoords = np.array([lms[i] for i in verts])
def get_limits(vertices, offset):
    xmin, xmax, ymin, ymax = np.min(vcoords[:, 0]), np.max(vcoords[:,0]), np.min(vcoords[:,1]), np.max(vcoords[:, 1])
    xspan, yspan = xmax-xmin, ymax-ymin
    return [[xmin - xspan*offset, xmax + xspan*offset], [ymin - yspan*offset, ymax + yspan*offset]]
limits = get_limits(vcoords, 0.2)

X,Y = np.mgrid[limits[0][0]:limits[0][1]:0.0005, limits[1][0]:limits[1][1]:0.0005]
xy = np.vstack((X.flatten(), Y.flatten())).T
grid_dists = cdist(xy, lms, 'euclidean')
points_dists = cdist(points, lms, 'euclidean')

def plot_areas_of_influence(verts, dim):
    sim_wareas = dict()
    sim_witnesses = dict()
    for sim in combinations(verts, dim+1):
        sim_wareas[tuple(np.sort(sim))] = []
        sim_witnesses[tuple(np.sort(sim))] = []
    for x_arg_sorted, x in zip(np.argsort(grid_dists, axis=1), range(len(xy))):
        wit_sim = tuple(np.sort(x_arg_sorted[:dim+1]))
        if wit_sim in sim_wareas:
            sim_wareas[wit_sim].append(x)
    for x_arg_sorted, x in zip(np.argsort(points_dists, axis=1), range(len(points))):
        wit_sim = tuple(np.sort(x_arg_sorted[:dim+1]))
        if wit_sim in sim_witnesses:
            sim_witnesses[wit_sim].append(x)
    return sim_wareas, sim_witnesses

# ###########
# ### Compute edge areas of influence
# ###########
# for edge in combinations(verts, 2):
#     edge_wareas[tuple(np.sort(edge))] = []
#     edge_witnesses[tuple(np.sort(edge))] = []
# for x_arg_sorted, x in zip(np.argsort(grid_dists, axis=1), range(len(xy))):
#     wit_edge = tuple(np.sort(x_arg_sorted[:2]))
#     if wit_edge in edge_wareas:
#         edge_wareas[wit_edge].append(x)
# for x_arg_sorted, x in zip(np.argsort(points_dists, axis=1), range(len(points))):
#     wit_edge = tuple(np.sort(x_arg_sorted[:2]))
#     if wit_edge in edge_witnesses:
#         edge_witnesses[wit_edge].append(x)

# fig = plt.figure()
# ax = plt.subplot()
# for points_list in edge_wareas.values():
#     if len(points_list) > 0:
#         e_xy = np.array([xy[i, :] for i in points_list])
#         ax.scatter(e_xy[:, 0], e_xy[:, 1])
# ax.scatter(lms[:, 0], lms[:, 1], color='k', s = 10.5)
# ax.scatter(points[:, 0], points[:, 1], color='k', s = 0.5)
# ax.set_aspect('equal')
# plt.xlim(limits[0])
# plt.ylim(limits[1])


fig = plt.figure(figsize=(8,8))
ax = plt.subplot()
sim_wareas, sim_witnesses = plot_areas_of_influence(verts, dim)
print([(s, len(sim_witnesses[s])) for s in sim_witnesses])
for sim in sim_wareas:
    points_list = sim_wareas[sim]
    if len(points_list) > 0:
        e_xy = np.array([xy[i, :] for i in points_list])
        ax.scatter(e_xy[:, 0], e_xy[:, 1], label=sim)
ax.scatter(lms[:, 0], lms[:, 1], color='k', s = 10.5)
ax.scatter(points[:, 0], points[:, 1], color='k', s = 0.5)
ax.set_aspect('equal')
for v in verts:
    ax.annotate(str(v), (lms[v, 0], lms[v, 1]), fontsize=15)
ax.legend()
plt.title(str(dim)+"-areas of influence")
plt.xlim(limits[0])
plt.ylim(limits[1])

plt.show()

In [ ]:
xy.shape

In [ ]:
### this is broken

fig = plt.figure()
ax = plt.subplot()
print(edge_witnesses)
for points_list in edge_witnesses.values():
    if len(points_list) > 0:
        e_xy = np.array([points[i, :] for i in points_list])
        ax.scatter(e_xy[:, 0], e_xy[:, 1])
ax.scatter(lms[:, 0], lms[:, 1], color='k', s = 10.5)
ax.scatter(points[:, 0], points[:, 1], color='k', s = 0.5)
ax.set_aspect('equal')
plt.xlim(limits[0])
plt.ylim(limits[1])
plt.show()

In [ ]:
## patched witness
fig = plt.figure()
ax = plt.subplot()
anc2 = AlphaNerveComplex(lms, eps, 2, points=points)
draw_complex(anc2, fig=fig, ax=ax)
plt.show()
